### 문화 행사 정보 

In [49]:
import pandas as pd
import requests
import time
import io
import os
from tqdm import tqdm

# 카카오 REST API 키 입력 (https://developers.kakao.com)
KAKAO_API_KEY = "api_key"  # 여기에 본인의 REST API 키 입력

CSV_FILE_PATH = "서울시_문화행사_정보.csv"  # 원본 CSV 경로

# 1. 한글 인코딩 오류 방지를 위한 robust CSV 읽기 함수
def read_csv_robust(file_path):
    encodings = ['cp949', 'utf-8', 'euc-kr', 'utf-8-sig']
    for encoding in encodings:
        try:
            print(f"⏳ '{encoding}' 인코딩 시도 중...")
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✅ 파일 읽기 성공: {encoding}")
            return df
        except Exception as e:
            print(f"❌ 실패 ({encoding}): {e}")
    raise ValueError("⚠️ 모든 인코딩 시도 실패")

# 2. 카카오 reverse geocoding 함수
def get_admin_code_kakao(lat, lng, api_key):
    url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"x": lng, "y": lat}
    try:
        res = requests.get(url, headers=headers, params=params, timeout=5)
        res.raise_for_status()
        data = res.json()
        for doc in data.get("documents", []):
            if doc['region_type'] == 'H':  # 행정동 우선
                return {
                    '행정동코드': doc['code'],
                    '시도': doc['region_1depth_name'],
                    '시군구': doc['region_2depth_name'],
                    '행정동': doc['region_3depth_name'],
                    '성공': True,
                    '오류': None
                }
        return {'행정동코드': None, '시도': None, '시군구': None, '행정동': None, '성공': False, '오류': 'No region_type H'}
    except Exception as e:
        return {'행정동코드': None, '시도': None, '시군구': None, '행정동': None, '성공': False, '오류': str(e)}

# 3. 메인 실행 함수
def main():
    df = read_csv_robust(CSV_FILE_PATH)
    print(f"총 {len(df)}개의 행이 있습니다.")

    # 위도/경도 컬럼 자동 탐지
    lat_col = next((col for col in df.columns if "위도" in col or "Y" in col or "lat" in col.lower()), None)
    lng_col = next((col for col in df.columns if "경도" in col or "X" in col or "lon" in col.lower()), None)
    if lat_col is None or lng_col is None:
        raise ValueError("⚠️ 위도/경도 컬럼을 찾을 수 없습니다.")

    # 날짜/시간 컬럼 자동 탐지
    date_col = next((col for col in df.columns if "일" in col or "날짜" in col), None)
    time_col = next((col for col in df.columns if "시" in col or "시간" in col), None)

    print(f"사용할 위도: {lat_col}, 경도: {lng_col}")
    print(f"사용할 날짜: {date_col}, 시간: {time_col}")

    # 결과 리스트 초기화
    행정동코드, 시도, 시군구, 행정동, 성공, 오류 = [], [], [], [], [], []

    print("🗺️ 좌표 → 행정동 정보 변환 중...")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        lat, lng = row[lat_col], row[lng_col]
        if pd.isna(lat) or pd.isna(lng):
            행정동코드.append(None); 시도.append(None); 시군구.append(None); 행정동.append(None)
            성공.append(False); 오류.append("Invalid coord")
            continue
        result = get_admin_code_kakao(lat, lng, KAKAO_API_KEY)
        행정동코드.append(result['행정동코드'])
        시도.append(result['시도'])
        시군구.append(result['시군구'])
        행정동.append(result['행정동'])
        성공.append(result['성공'])
        오류.append(result['오류'])
        time.sleep(0.1)  # 카카오 초당 10회 제한

    # 결과 결합
    df["행정동코드"] = 행정동코드
    df["시도"] = 시도
    df["시군구"] = 시군구
    df["행정동"] = 행정동
    df["geocoding_success"] = 성공
    df["geocoding_error"] = 오류

    # 필요한 컬럼만 저장
    save_cols = [col for col in [date_col, time_col, lat_col, lng_col, "행정동코드", "시도", "시군구", "행정동"] if col]
    df_result = df[save_cols]

    output_file = "서울시_문화행사_위치정보_시간포함.csv"
    df_result.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"\n✅ 최종 CSV 저장 완료: {output_file}")

if __name__ == "__main__":
    main()


  0%|          | 0/5597 [00:00<?, ?it/s]

⏳ 'cp949' 인코딩 시도 중...
❌ 실패 (cp949): 'cp949' codec can't decode byte 0x98 in position 8: illegal multibyte sequence
⏳ 'utf-8' 인코딩 시도 중...
✅ 파일 읽기 성공: utf-8
총 5597개의 행이 있습니다.
사용할 위도: 위도(Y좌표), 경도: 경도(X좌표)
사용할 날짜: 날짜/시간, 시간: 날짜/시간
🗺️ 좌표 → 행정동 정보 변환 중...


100%|██████████| 5597/5597 [16:47<00:00,  5.56it/s]


✅ 최종 CSV 저장 완료: 서울시_문화행사_위치정보_시간포함.csv


In [2]:
import pandas as pd

# 가장 완성된 문화행사 데이터 로드
culture_df = pd.read_csv("서울시_문화행사_위치정보_시간포함.csv", encoding='utf-8-sig')
print(f"로드된 데이터: {len(culture_df)}개")
print(f"컬럼: {culture_df.columns.tolist()}")

# 데이터 확인
print(culture_df.head())


로드된 데이터: 5597개
컬럼: ['날짜/시간', '날짜/시간.1', '위도(Y좌표)', '경도(X좌표)', '행정동코드', '시도', '시군구', '행정동']
                   날짜/시간                날짜/시간.1    위도(Y좌표)     경도(X좌표)  \
0  2025-12-18~2025-12-21  2025-12-18~2025-12-21  37.511824  127.059159   
1  2025-12-06~2025-12-06  2025-12-06~2025-12-06  37.549906  126.945534   
2  2025-10-18~2025-10-19  2025-10-18~2025-10-19  37.457066  126.896037   
3  2025-10-03~2025-10-12  2025-10-03~2025-10-12  37.529365  127.073978   
4  2025-09-27~2025-09-28  2025-09-27~2025-09-28  37.641994  127.077437   

          행정동코드     시도  시군구   행정동  
0  1.168058e+09  서울특별시  강남구  삼성1동  
1  1.144060e+09  서울특별시  마포구   대흥동  
2  1.154567e+09  서울특별시  금천구  시흥1동  
3  1.121584e+09  서울특별시  광진구  자양3동  
4  1.135061e+09  서울특별시  노원구  하계1동  


In [7]:
import pandas as pd
import chardet

# 파일의 실제 인코딩 확인
def detect_file_encoding(file_path):
    with open(file_path, 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        return result

# 인코딩 확인
encoding_info = detect_file_encoding("서울시_문화행사_정보.csv")
print(f"감지된 인코딩: {encoding_info}")

# 감지된 인코딩으로 파일 읽기
df = pd.read_csv("서울시_문화행사_정보.csv", encoding=encoding_info['encoding'])


감지된 인코딩: {'encoding': 'UTF-8-SIG', 'confidence': 1.0, 'language': ''}


In [8]:
import pandas as pd

# 1. 원본 파일을 UTF-8-SIG 인코딩으로 읽기
print("UTF-8-SIG 인코딩으로 원본 파일 읽는 중...")
original_df = pd.read_csv("서울시_문화행사_정보.csv", encoding='UTF-8-SIG')

print(f"원본 데이터: {len(original_df)}개")
print(f"컬럼명: {original_df.columns.tolist()}")

# 2. 분류와 유무료 컬럼 확인
print(f"\n분류 컬럼 샘플: {original_df.iloc[:5, 0].tolist()}")  # 0번째 컬럼
print(f"유무료 컬럼 샘플: {original_df.iloc[:5, 20].tolist()}")  # 20번째 컬럼

# 3. 현재 문화행사 데이터 읽기
culture_df = pd.read_csv("서울시_문화행사_위치정보_시간포함.csv", encoding='utf-8-sig')

# 4. 원본에서 필요한 정보 추출
original_info = pd.DataFrame({
    'category': original_df.iloc[:, 0],      # 분류 (0번째 컬럼)
    'free_paid': original_df.iloc[:, 20],    # 유무료 (20번째 컬럼)
    'latitude': original_df.iloc[:, 18],     # 위도 (18번째 컬럼)
    'longitude': original_df.iloc[:, 19]     # 경도 (19번째 컬럼)
})

# 5. 매칭하여 분류/유무료 정보 추가
culture_df['분류'] = ''
culture_df['유무료'] = ''

match_count = 0

print("위도, 경도 기준으로 분류/유무료 정보 매칭 중...")

for idx, row in culture_df.iterrows():
    current_lat = row['위도(Y좌표)']
    current_lng = row['경도(X좌표)']
    
    # 매칭되는 원본 데이터 찾기
    matching_rows = original_info[
        (abs(original_info['latitude'] - current_lat) < 0.00001) &
        (abs(original_info['longitude'] - current_lng) < 0.00001)
    ]
    
    if len(matching_rows) > 0:
        match = matching_rows.iloc[0]
        culture_df.at[idx, '분류'] = str(match['category'])
        culture_df.at[idx, '유무료'] = str(match['free_paid'])
        match_count += 1

print(f"\n매칭 성공: {match_count}개")

# 6. 결과 확인
print(f"\n분류별 통계:")
category_counts = culture_df['분류'].value_counts()
print(category_counts.head(10))

print(f"\n유무료 통계:")
fee_counts = culture_df['유무료'].value_counts()
print(fee_counts)

# 7. 최종 파일 저장
output_file = "서울시_문화행사_최종완성.csv"
culture_df.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n✅ 최종 완성된 파일이 '{output_file}'로 저장되었습니다.")

# 8. 결과 미리보기
print(f"\n최종 데이터 미리보기:")
print(culture_df[['날짜/시간', '분류', '유무료', '행정동코드', '시도', '시군구', '행정동']].head(10))


UTF-8-SIG 인코딩으로 원본 파일 읽는 중...
원본 데이터: 5597개
컬럼명: ['분류', '자치구', '공연/행사명', '날짜/시간', '장소', '기관명', '이용대상', '이용요금', '출연자정보', '프로그램소개', '기타내용', '홈페이지?주소', '대표이미지', '신청일', '시민/기관', '시작일', '종료일', '테마분류', '위도(Y좌표)', '경도(X좌표)', '유무료', '문화포털상세URL']

분류 컬럼 샘플: ['전시/미술', '클래식', '축제-문화/예술', '축제-문화/예술', '축제-문화/예술']
유무료 컬럼 샘플: ['유료', '유료', '무료', '무료', '무료']
위도, 경도 기준으로 분류/유무료 정보 매칭 중...

매칭 성공: 5596개

분류별 통계:
교육/체험       2240
클래식         1183
축제-문화/예술     431
전시/미술        411
콘서트          351
연극           264
영화           190
기타           165
국악           101
뮤지컬/오페라       92
Name: 분류, dtype: int64

유무료 통계:
무료    3719
유료    1877
         1
Name: 유무료, dtype: int64

✅ 최종 완성된 파일이 '서울시_문화행사_최종완성.csv'로 저장되었습니다.

최종 데이터 미리보기:
                   날짜/시간        분류 유무료         행정동코드     시도   시군구   행정동
0  2025-12-18~2025-12-21     전시/미술  유료  1.168058e+09  서울특별시   강남구  삼성1동
1  2025-12-06~2025-12-06       클래식  유료  1.144060e+09  서울특별시   마포구   대흥동
2  2025-10-18~2025-10-19  축제-문화/예술  무료  1.154567e+09  서울특별시   금천구  시흥1

In [12]:
import os

# 모든 파일 목록 확인
all_files = os.listdir('.')
print("현재 디렉토리의 모든 파일:")
for file in all_files:
    print(f"  - {file}")

# 문화행사 관련 파일 찾기
culture_files = [f for f in all_files if '문화' in f or 'culture' in f.lower()]
print(f"\n문화행사 관련 파일: {culture_files}")


현재 디렉토리의 모든 파일:
  - .ipynb_checkpoints
  - data merging.ipynb
  - 병합된_데이터셋_문화행사_상세.csv
  - 서울시_문화행사_위치정보_시간포함.csv
  - 서울시_문화행사_정보.csv
  - 서울시_문화행사_최종완성.csv

문화행사 관련 파일: ['병합된_데이터셋_문화행사_상세.csv', '서울시_문화행사_위치정보_시간포함.csv', '서울시_문화행사_정보.csv', '서울시_문화행사_최종완성.csv']


In [16]:
import os

# 현재 디렉토리의 모든 파일 확인
all_files = os.listdir('.')
print("현재 디렉토리의 모든 파일:")
for file in all_files:
    print(f"  - {file}")

# CSV 파일만 필터링
csv_files = [f for f in all_files if f.endswith('.csv')]
print(f"\nCSV 파일들: {csv_files}")

# 문화행사 관련 파일 찾기
culture_files = [f for f in all_files if '문화' in f or 'culture' in f.lower()]
print(f"문화행사 관련 파일: {culture_files}")


현재 디렉토리의 모든 파일:
  - .ipynb_checkpoints
  - data merging.ipynb
  - merged_holidays.csv
  - 병합된_데이터셋_문화행사_상세.csv
  - 서울시_문화행사_위치정보_시간포함.csv
  - 서울시_문화행사_정보.csv
  - 서울시_문화행사_최종완성_수정.csv

CSV 파일들: ['merged_holidays.csv', '병합된_데이터셋_문화행사_상세.csv', '서울시_문화행사_위치정보_시간포함.csv', '서울시_문화행사_정보.csv', '서울시_문화행사_최종완성_수정.csv']
문화행사 관련 파일: ['병합된_데이터셋_문화행사_상세.csv', '서울시_문화행사_위치정보_시간포함.csv', '서울시_문화행사_정보.csv', '서울시_문화행사_최종완성_수정.csv']


In [19]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def parse_date_range(date_str):
    """
    날짜 범위 문자열을 파싱하여 시작일과 종료일을 반환
    예: "2025-06-28~2025-06-28" -> (datetime(2025,6,28), datetime(2025,6,28))
    """
    if pd.isna(date_str) or date_str == '':
        return None, None
    
    try:
        date_str = str(date_str).strip()
        if '~' in date_str:
            start_str, end_str = date_str.split('~')
            start_date = pd.to_datetime(start_str.strip())
            end_date = pd.to_datetime(end_str.strip())
            return start_date, end_date
        else:
            single_date = pd.to_datetime(date_str)
            return single_date, single_date
    except Exception as e:
        print(f"날짜 파싱 오류: {date_str} - {e}")
        return None, None

def merge_culture_with_local_people():
    """
    문화행사 데이터를 local people dong 데이터셋과 날짜 범위 기반으로 병합
    """
    print("데이터 로드 중...")
    
    # 1. local people dong 데이터 로드
    local_people_df = pd.read_csv("LOCAL_PEOPLE_DONG_202405_202504_filtered_max.csv")
    print(f"local people dong 데이터: {len(local_people_df)}개 행")
    
    # 2. 문화행사 데이터 로드
    culture_df = pd.read_csv("서울시_문화행사_최종완성_수정.csv", encoding='utf-8-sig')
    print(f"문화행사 데이터: {len(culture_df)}개 행사")
    
    # 3. local people dong 데이터의 날짜 컬럼을 datetime으로 변환
    local_people_df['date'] = pd.to_datetime(local_people_df['STDR_DE_ID'], format='%Y%m%d')
    
    # 4. 행정동코드 형식 통일 (8자리)
    local_people_df['admin_code_8'] = local_people_df['ADSTRD_CODE_SE'].astype(str).str[:8]
    culture_df['admin_code_8'] = culture_df['행정동코드'].astype(str).str[:8]
    
    # 5. local people dong에 있는 행정동코드만 추출 (효율성을 위해)
    valid_admin_codes = set(local_people_df['admin_code_8'].unique())
    print(f"local people dong에 있는 고유 행정동코드: {len(valid_admin_codes)}개")
    
    # 6. 문화행사 데이터를 valid_admin_codes로 필터링
    culture_filtered = culture_df[culture_df['admin_code_8'].isin(valid_admin_codes)]
    print(f"필터링된 문화행사 데이터: {len(culture_filtered)}개 (원본: {len(culture_df)}개)")
    
    if len(culture_filtered) == 0:
        print("⚠️ local people dong과 매칭되는 문화행사 데이터가 없습니다.")
        # 기본 컬럼들을 추가
        local_people_df['has_culture_event'] = 0
        local_people_df['culture_event_count'] = 0
        local_people_df['분류'] = ''
        local_people_df['유무료'] = ''
        return local_people_df
    
    # 7. 날짜 범위 기반 매칭을 위한 일별 레코드 생성
    print("날짜 범위 파싱 및 일별 레코드 생성 중...")
    daily_culture_records = []
    
    for idx, culture_row in culture_filtered.iterrows():
        start_date, end_date = parse_date_range(culture_row['날짜/시간'])
        
        if start_date is None or end_date is None:
            continue
            
        admin_code = culture_row['admin_code_8']
        
        # 해당 기간의 모든 날짜에 대해 레코드 생성
        current_date = start_date
        while current_date <= end_date:
            daily_culture_records.append({
                'date': current_date,
                'admin_code_8': admin_code,
                '분류': culture_row.get('분류', ''),
                '유무료': culture_row.get('유무료', ''),
                'has_culture_event': 1
            })
            current_date += timedelta(days=1)
    
    # 8. 일별 문화행사 DataFrame 생성
    daily_culture_df = pd.DataFrame(daily_culture_records)
    
    if len(daily_culture_df) == 0:
        print("⚠️ 매칭 가능한 문화행사 데이터가 없습니다.")
        local_people_df['has_culture_event'] = 0
        local_people_df['culture_event_count'] = 0
        local_people_df['분류'] = ''
        local_people_df['유무료'] = ''
        return local_people_df
    
    print(f"생성된 일별 문화행사 레코드: {len(daily_culture_df)}개")
    
    # 9. local people dong의 날짜 범위 내에 있는 문화행사만 필터링
    min_date = local_people_df['date'].min()
    max_date = local_people_df['date'].max()
    daily_culture_df = daily_culture_df[
        (daily_culture_df['date'] >= min_date) & 
        (daily_culture_df['date'] <= max_date)
    ]
    print(f"날짜 범위 필터링 후: {len(daily_culture_df)}개")
    
    # 10. 날짜와 행정동코드별로 문화행사 집계
    culture_aggregated = daily_culture_df.groupby(['date', 'admin_code_8']).agg({
        'has_culture_event': 'sum',  # 해당 날짜/지역의 문화행사 개수
        '분류': lambda x: '|'.join(x.unique()),  # 문화행사 분류들 (중복 제거)
        '유무료': lambda x: '|'.join(x.unique())  # 유무료 정보들 (중복 제거)
    }).reset_index()
    
    # 11. 컬럼명 정리
    culture_aggregated.rename(columns={
        'has_culture_event': 'culture_event_count'
    }, inplace=True)
    
    # 12. local people dong 데이터와 병합
    print("데이터 병합 수행 중...")
    merged_df = local_people_df.merge(
        culture_aggregated,
        left_on=['date', 'admin_code_8'],
        right_on=['date', 'admin_code_8'],
        how='left'
    )
    
    # 13. 문화행사가 없는 경우 기본값으로 채우기
    merged_df['culture_event_count'] = merged_df['culture_event_count'].fillna(0).astype(int)
    merged_df['has_culture_event'] = (merged_df['culture_event_count'] > 0).astype(int)
    merged_df['분류'] = merged_df['분류'].fillna('')  # 빈 문자열로 채움
    merged_df['유무료'] = merged_df['유무료'].fillna('')  # 빈 문자열로 채움
    
    # 14. 임시 컬럼 제거
    merged_df = merged_df.drop('admin_code_8', axis=1)
    
    # 15. 결과 통계
    total_records = len(merged_df)
    records_with_events = len(merged_df[merged_df['has_culture_event'] == 1])
    total_events = merged_df['culture_event_count'].sum()
    
    print(f"\n📊 병합 결과:")
    print(f"전체 레코드: {total_records:,}개")
    print(f"문화행사가 있는 레코드: {records_with_events:,}개 ({records_with_events/total_records*100:.2f}%)")
    print(f"총 문화행사 발생: {total_events:,}건")
    
    # 16. 분류별 통계
    if records_with_events > 0:
        print(f"\n📈 문화행사 분류별 상위 10개:")
        category_stats = merged_df[merged_df['has_culture_event'] == 1]['분류'].value_counts().head(10)
        for category, count in category_stats.items():
            print(f"  {category}: {count}건")
        
        print(f"\n💰 유무료 통계:")
        fee_stats = merged_df[merged_df['has_culture_event'] == 1]['유무료'].value_counts()
        for fee_type, count in fee_stats.items():
            print(f"  {fee_type}: {count}건")
    
    return merged_df

# 실행
try:
    final_df = merge_culture_with_local_people()
    
    # 결과 저장
    output_file = "LOCAL_PEOPLE_DONG_with_culture.csv"
    final_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n💾 병합된 데이터가 '{output_file}'로 저장되었습니다.")
    
    # 결과 미리보기
    print(f"\n📋 새로 추가된 문화행사 관련 컬럼:")
    culture_columns = ['has_culture_event', 'culture_event_count', '분류', '유무료']
    print(f"  {culture_columns}")
    
    print(f"\n📋 최종 데이터 컬럼:")
    print(final_df.columns.tolist())
    
    print(f"\n📋 문화행사가 있는 샘플 데이터:")
    sample_with_culture = final_df[final_df['has_culture_event'] == 1].head(5)
    display_cols = ['STDR_DE_ID', 'ADSTRD_CODE_SE', 'TOT_LVPOP_CO', 'culture_event_count', '분류', '유무료']
    available_cols = [col for col in display_cols if col in final_df.columns]
    if len(sample_with_culture) > 0:
        print(sample_with_culture[available_cols])
    
except FileNotFoundError as e:
    print(f"❌ 파일을 찾을 수 없습니다: {e}")
    print("다음 파일들이 필요합니다:")
    print("1. LOCAL_PEOPLE_DONG_202405_202504_filtered_max.csv")
    print("2. 서울시_문화행사_최종완성_수정.csv")
except Exception as e:
    print(f"❌ 오류 발생: {e}")
    import traceback
    traceback.print_exc()


데이터 로드 중...
local people dong 데이터: 5840개 행
문화행사 데이터: 5597개 행사
local people dong에 있는 고유 행정동코드: 16개
필터링된 문화행사 데이터: 627개 (원본: 5597개)
날짜 범위 파싱 및 일별 레코드 생성 중...
생성된 일별 문화행사 레코드: 11047개
날짜 범위 필터링 후: 7925개
데이터 병합 수행 중...

📊 병합 결과:
전체 레코드: 5,840개
문화행사가 있는 레코드: 2,319개 (39.71%)
총 문화행사 발생: 7,925건

📈 문화행사 분류별 상위 10개:
  전시/미술: 790건
  클래식: 168건
  교육/체험: 134건
  교육/체험|전시/미술: 79건
  교육/체험|연극: 71건
  연극|교육/체험: 66건
  축제-시민화합: 66건
  연극: 59건
  연극|전시/미술: 51건
  교육/체험|뮤지컬/오페라: 50건

💰 유무료 통계:
  무료: 1292건
  유료: 481건
  무료|유료: 412건
  유료|무료: 134건

💾 병합된 데이터가 'LOCAL_PEOPLE_DONG_with_culture.csv'로 저장되었습니다.

📋 새로 추가된 문화행사 관련 컬럼:
  ['has_culture_event', 'culture_event_count', '분류', '유무료']

📋 최종 데이터 컬럼:
['STDR_DE_ID', 'TMZON_PD_SE', 'ADSTRD_CODE_SE', 'TOT_LVPOP_CO', 'date', 'culture_event_count', '분류', '유무료', 'has_culture_event']

📋 문화행사가 있는 샘플 데이터:
     STDR_DE_ID  ADSTRD_CODE_SE  TOT_LVPOP_CO  culture_event_count  \
480    20240531        11110615   127308.8305                    1   
481    20240531        11110650   

#### 공휴일 데이터 수정

In [21]:
import pandas as pd
import numpy as np
from datetime import datetime

def merge_holidays_with_corrected_data():
    """
    merged_holidays.csv의 is_holiday를 수정하여 LOCAL_PEOPLE_DONG_with_culture.csv에 병합
    """
    print("데이터 로드 중...")
    
    # 1. 데이터 로드
    holidays_df = pd.read_csv("merged_holidays.csv")
    culture_df = pd.read_csv("LOCAL_PEOPLE_DONG_with_culture.csv")
    
    # 2. 2024-2025년 법정공휴일 정의 (누락된 것들 포함)
    holidays_2024 = [
        20240101,  # 신정
        20240209, 20240210, 20240211, 20240212,  # 설날 연휴
        20240301,  # 삼일절
        20240410,  # 국회의원선거일
        20240505, 20240506,  # 어린이날
        20240515,  # 부처님오신날
        20240606,  # 현충일
        20240815,  # 광복절
        20240916, 20240917, 20240918,  # 추석 연휴
        20241003,  # 개천절
        20241009,  # 한글날
        20241225,  # 크리스마스 ⭐ 누락된 공휴일
    ]
    
    holidays_2025 = [
        20250101,  # 신정
        20250128, 20250129, 20250130,  # 설날 연휴
        20250301, 20250303,  # 삼일절
        20250505, 20250506,  # 어린이날, 부처님오신날
        20250606,  # 현충일
        20250815,  # 광복절
        20251003,  # 개천절
        20251006, 20251007, 20251008, 20251009,  # 추석 연휴, 한글날
        20251225,  # 크리스마스
    ]
    
    all_holidays = holidays_2024 + holidays_2025
    
    # 3. 날짜 형식 통일
    holidays_df['date_key'] = holidays_df['date'].astype(str)
    culture_df['date_key'] = culture_df['STDR_DE_ID'].astype(str)
    
    # 4. 행정동코드 형식 통일
    holidays_df['admin_key'] = holidays_df['ADSTRD_CODE_SE'].astype(str)
    culture_df['admin_key'] = culture_df['ADSTRD_CODE_SE'].astype(str)
    
    # 5. ⭐ 공휴일 정보 수정 (기존 is_holiday 무시하고 새로 계산)
    holidays_df['is_holiday_corrected'] = holidays_df['date'].isin(all_holidays).astype(int)
    
    # 6. 수정 전후 비교
    original_holidays = holidays_df['is_holiday'].sum()
    corrected_holidays = holidays_df['is_holiday_corrected'].sum()
    print(f"수정 전 공휴일 레코드: {original_holidays:,}개")
    print(f"수정 후 공휴일 레코드: {corrected_holidays:,}개")
    print(f"차이: {corrected_holidays - original_holidays:,}개")
    
    # 7. 크리스마스 확인
    christmas_2024 = holidays_df[holidays_df['date'] == 20241225]['is_holiday_corrected'].iloc[0] if len(holidays_df[holidays_df['date'] == 20241225]) > 0 else "데이터 없음"
    print(f"2024-12-25 크리스마스 수정 후: {christmas_2024}")
    
    # 8. 수정된 공휴일 정보로 병합
    holiday_info = holidays_df[['date_key', 'admin_key', 'is_holiday_corrected']].drop_duplicates()
    holiday_info.rename(columns={'is_holiday_corrected': 'is_holiday'}, inplace=True)
    
    # 9. LOCAL_PEOPLE_DONG_with_culture와 병합
    print("수정된 공휴일 정보 병합 중...")
    merged_df = culture_df.merge(
        holiday_info,
        left_on=['date_key', 'admin_key'],
        right_on=['date_key', 'admin_key'],
        how='left'
    )
    
    # 10. 병합되지 않은 경우 기본값 설정
    merged_df['is_holiday'] = merged_df['is_holiday'].fillna(0).astype(int)
    
    # 11. 임시 컬럼 제거
    merged_df = merged_df.drop(['date_key', 'admin_key'], axis=1)
    
    # 12. 결과 통계
    total_records = len(merged_df)
    holiday_records = len(merged_df[merged_df['is_holiday'] == 1])
    
    print(f"\n📊 최종 병합 결과:")
    print(f"전체 레코드: {total_records:,}개")
    print(f"공휴일 레코드: {holiday_records:,}개 ({holiday_records/total_records*100:.2f}%)")
    
    # 13. 결과 저장
    output_file = "LOCAL_PEOPLE_DONG_final_corrected.csv"
    merged_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"\n💾 수정된 공휴일 정보가 포함된 최종 데이터가 '{output_file}'로 저장되었습니다.")
    
    return merged_df

# 실행
final_df = merge_holidays_with_corrected_data()


데이터 로드 중...
수정 전 공휴일 레코드: 192개
수정 후 공휴일 레코드: 272개
차이: 80개
2024-12-25 크리스마스 수정 후: 1
수정된 공휴일 정보 병합 중...

📊 최종 병합 결과:
전체 레코드: 5,840개
공휴일 레코드: 272개 (4.66%)

💾 수정된 공휴일 정보가 포함된 최종 데이터가 'LOCAL_PEOPLE_DONG_final_corrected.csv'로 저장되었습니다.


#### 날씨 데이터

In [24]:
import pandas as pd

# 유동인구 데이터 불러오기
people_df = pd.read_csv("LOCAL_PEOPLE_DONG_final_corrected.csv")
people_df['date'] = pd.to_datetime(people_df['STDR_DE_ID'].astype(str), format="%Y%m%d")

# 날씨 데이터 불러오기
weather_df = pd.read_csv("OBS_ASOS_DD_20250531150815.csv", encoding="cp949")
weather_df['date'] = pd.to_datetime(weather_df['일시'], format="%Y-%m-%d")

# 날씨 feature 추출 및 컬럼명 영어로
weather_selected = weather_df[[
    'date', '평균기온(°C)', '최고기온(°C)', '최저기온(°C)',
    '일강수량(mm)', '평균 풍속(m/s)', '합계 일조시간(hr)',
    '평균 상대습도(%)', '평균 지면온도(°C)'
]].copy()

weather_selected.columns = [
    'date', 'avg_temp', 'max_temp', 'min_temp',
    'precipitation', 'wind_speed', 'sunshine_hours',
    'humidity', 'ground_temp'
]

# 병합
merged_df = pd.merge(people_df, weather_selected, on='date', how='left')

# ✅ 한글 컬럼/값 보존하면서 저장 (Excel 등에서 깨짐 방지)
merged_df.to_csv("merged_people_weather_utf8sig.csv", index=False, encoding="utf-8-sig")

print("✅ 저장 완료: 'merged_people_weather_utf8sig.csv' (한글 유지 + 깨짐 없음)")


✅ 저장 완료: 'merged_people_weather_utf8sig.csv' (한글 유지 + 깨짐 없음)


### 지하철 승하차 교통정보

In [10]:
def fix_subway_data_structure():
    """
    뒤섞인 지하철 데이터 구조를 올바르게 재구성
    """
    print("🔧 지하철 데이터 구조 수정")
    print("="*50)
    
    # 1. 원본 데이터 로드
    df = pd.read_csv("subway/CARD_SUBWAY_MONTH_202405.csv", 
                    encoding='utf-8-sig',
                    error_bad_lines=False)
    
    print(f"원본 데이터: {df.shape}")
    
    # 2. 올바른 컬럼 매핑으로 재구성
    df_fixed = pd.DataFrame({
        '사용일자': df.index,  # 인덱스가 실제 날짜인 것 같음
        '노선명': df['사용일자'],  # 첫 번째 컬럼이 실제 노선명
        '역명': df['노선명'],     # 두 번째 컬럼이 실제 역명
        '승차총승객수': df['역명'],    # 세 번째 컬럼이 실제 승차승객수
        '하차총승객수': df['승차총승객수'], # 네 번째 컬럼이 실제 하차승객수
        '등록일자': df['하차총승객수']   # 다섯 번째 컬럼이 실제 등록일자
    })
    
    print(f"\n📋 수정된 데이터 샘플:")
    print(df_fixed.head(10))
    
    # 3. 실제 날짜 추출 (인덱스에서)
    # 첫 번째 행의 인덱스가 20240501이므로 이를 활용
    first_date = 20240501
    df_fixed['사용일자'] = first_date  # 모든 행이 같은 날짜
    
    print(f"\n📋 날짜 수정 후:")
    print(df_fixed.head())
    
    # 4. 역명 확인
    print(f"\n📋 실제 역명 샘플 (처음 20개):")
    unique_stations = df_fixed['역명'].unique()
    for i, station in enumerate(unique_stations[:20]):
        print(f"  {i+1:2d}. {station}")
    
    return df_fixed

# 실행
fixed_subway_data = fix_subway_data_structure()


🔧 지하철 데이터 구조 수정
원본 데이터: (19110, 6)

📋 수정된 데이터 샘플:
              사용일자  노선명        역명  승차총승객수  하차총승객수      등록일자
20240501  20240501  안산선        안산   10467    9980  20240504
20240501  20240501  안산선        초지    4052    4244  20240504
20240501  20240501  안산선        고잔    6923    6696  20240504
20240501  20240501  안산선        중앙   17412   17710  20240504
20240501  20240501  안산선       한대앞    9509    9460  20240504
20240501  20240501  안산선       상록수   12528   12649  20240504
20240501  20240501  안산선        반월    3871    3879  20240504
20240501  20240501  안산선       대야미    4648    4154  20240504
20240501  20240501  안산선        산본   14497   15009  20240504
20240501  20240501  신림선  관악산(서울대)    4358    4908  20240504

📋 날짜 수정 후:
              사용일자  노선명   역명  승차총승객수  하차총승객수      등록일자
20240501  20240501  안산선   안산   10467    9980  20240504
20240501  20240501  안산선   초지    4052    4244  20240504
20240501  20240501  안산선   고잔    6923    6696  20240504
20240501  20240501  안산선   중앙   17412   17710  20240504
202

In [12]:
def fix_202502_encoding_issue():
    """
    2025년 2월 파일 인코딩 문제 완전 해결
    """
    print("🔧 2025년 2월 파일 인코딩 문제 해결")
    print("="*50)
    
    file_path = "subway/CARD_SUBWAY_MONTH_202502.csv"
    
    # 1. 바이너리 모드로 파일 읽어서 BOM 확인
    try:
        with open(file_path, 'rb') as f:
            raw_bytes = f.read(10)
            print(f"파일 시작 바이트: {raw_bytes}")
            
            # BOM 패턴 확인
            if raw_bytes.startswith(b'\xef\xbb\xbf'):
                print("✅ UTF-8 BOM 감지")
                encoding = 'utf-8-sig'
            elif raw_bytes.startswith(b'\xff\xfe'):
                print("✅ UTF-16 LE BOM 감지")
                encoding = 'utf-16'
            elif raw_bytes.startswith(b'\xfe\xff'):
                print("✅ UTF-16 BE BOM 감지")
                encoding = 'utf-16'
            else:
                print("❓ BOM 없음, 다양한 인코딩 시도")
                encoding = None
                
    except Exception as e:
        print(f"❌ 파일 읽기 실패: {e}")
        return None
    
    # 2. 다양한 인코딩 시도
    encodings_to_try = [
        'utf-8-sig', 'utf-8', 'euc-kr', 'cp949', 'latin1', 
        'iso-8859-1', 'utf-16', 'utf-16le', 'utf-16be'
    ]
    
    if encoding:
        encodings_to_try.insert(0, encoding)
    
    for enc in encodings_to_try:
        try:
            print(f"시도 중: {enc}")
            
            # 기본 로드 시도
            df = pd.read_csv(file_path, encoding=enc)
            print(f"✅ {enc} 인코딩으로 성공!")
            print(f"데이터 크기: {df.shape}")
            print(f"컬럼: {df.columns.tolist()}")
            return df
            
        except UnicodeDecodeError as e:
            print(f"❌ {enc}: 유니코드 오류 - {str(e)[:50]}...")
            continue
        except Exception as e:
            print(f"❌ {enc}: 기타 오류 - {str(e)[:50]}...")
            continue
    
    # 3. 마지막 시도: 오류 무시하고 로드
    try:
        print("\n🔄 오류 무시 모드로 시도...")
        df = pd.read_csv(file_path, 
                        encoding='utf-8', 
                        errors='ignore',
                        on_bad_lines='skip')
        print(f"✅ 오류 무시 모드 성공!")
        print(f"데이터 크기: {df.shape}")
        return df
        
    except Exception as e:
        print(f"❌ 오류 무시 모드도 실패: {e}")
    
    # 4. 최후의 수단: 텍스트 에디터로 확인 필요
    print("\n💡 해결 방법:")
    print("1. 파일을 메모장이나 텍스트 에디터로 열어서 인코딩 확인")
    print("2. 다른 인코딩으로 저장 후 재시도")
    print("3. 또는 해당 월 데이터를 다시 다운로드")
    
    return None

# 실행
feb_data = fix_202502_encoding_issue()

if feb_data is not None:
    print(f"\n📋 2월 데이터 샘플:")
    print(feb_data.head())


🔧 2025년 2월 파일 인코딩 문제 해결
파일 시작 바이트: b'\xbb\xe7\xbf\xeb\xc0\xcf\xc0\xda,\xb3'
❓ BOM 없음, 다양한 인코딩 시도
시도 중: utf-8-sig
❌ utf-8-sig: 유니코드 오류 - 'utf-8' codec can't decode byte 0xbb in position 0...
시도 중: utf-8
❌ utf-8: 유니코드 오류 - 'utf-8' codec can't decode byte 0xbb in position 0...
시도 중: euc-kr
✅ euc-kr 인코딩으로 성공!
데이터 크기: (17277, 6)
컬럼: ['사용일자', '노선명', '역명', '승차총승객수', '하차총승객수', '등록일자']

📋 2월 데이터 샘플:
       사용일자  노선명    역명  승차총승객수  하차총승객수      등록일자
0  20250201  1호선   서울역   56802   45819  20250204
1  20250201  1호선    시청   31681   29724  20250204
2  20250201  1호선    종각   27189   25388  20250204
3  20250201  1호선  종로3가   25776   23449  20250204
4  20250201  1호선  종로5가   20211   20055  20250204


In [13]:
def process_complete_12_months_subway_data():
    """
    2월 포함 완전한 12개월 지하철 데이터 처리
    """
    print("🚇 완전한 12개월 지하철 데이터 처리")
    print("="*50)
    
    # 조사 대상 행정동 지하철역 매핑
    station_mapping = {
        '종각': '11110615', '종로3가': '11110615', '광화문': '11110615',
        '을지로3가': '11110615', '종로5가': '11110615', '동묘앞': '11110650',
        '명동': '11140550', '을지로입구': '11140550', '을지로4가': '11140550',
        '동대문': '11140605', '동대문역사문화공원': '11140605',
        '용산': '11170520', '한강진': '11170520', '삼각지': '11170520',
        '이태원': '11170650', '뚝섬': '11200660', '건대입구': '11200660',
        '성수': '11200690', '합정': '11440660', '상수': '11440690',
        '홍대입구': '11440710', '망원': '11440710', '신촌': '11440710',
        '여의도': '11560605', '당산': '11560605', '영등포구청': '11560605',
        '신림': '11620745', '봉천': '11620745', '강남': '11680510',
        '역삼': '11680640', '선릉': '11680640', '압구정': '11680640',
        '잠실': '11710580', '석촌': '11710580'
    }
    
    # 모든 월별 파일 처리 (2월 포함)
    subway_files = [
        ("subway/CARD_SUBWAY_MONTH_202405.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202406.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202407.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202408.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202409.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202410.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202411.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202412.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202501.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202502.csv", 'euc-kr'),  # 2월은 EUC-KR
        ("subway/CARD_SUBWAY_MONTH_202503.csv", 'utf-8-sig'),
        ("subway/CARD_SUBWAY_MONTH_202504.csv", 'utf-8-sig')
    ]
    
    all_subway_data = []
    
    for file_path, encoding in subway_files:
        try:
            # 파일별 적절한 인코딩으로 로드
            df = pd.read_csv(file_path, encoding=encoding, error_bad_lines=False)
            
            # 2월 데이터는 이미 올바른 구조
            if '202502' in file_path:
                df_processed = df.copy()
                # 날짜 컬럼이 이미 올바름
            else:
                # 다른 월들은 구조 재정렬 필요
                df_processed = pd.DataFrame({
                    '사용일자': df.index.map(lambda x: int(str(x)[:8]) if len(str(x)) >= 8 else 20240501),
                    '노선명': df['사용일자'],
                    '역명': df['노선명'],
                    '승차총승객수': df['역명'],
                    '하차총승객수': df['승차총승객수'],
                    '등록일자': df['하차총승객수']
                })
            
            # 조사 대상 역만 필터링
            target_stations = list(station_mapping.keys())
            df_target = df_processed[df_processed['역명'].isin(target_stations)].copy()
            
            # 행정동코드 추가
            df_target['행정동코드'] = df_target['역명'].map(station_mapping)
            
            if len(df_target) > 0:
                all_subway_data.append(df_target)
                print(f"✅ {file_path}: {len(df_target)}개 행 추출")
            else:
                print(f"❌ {file_path}: 조사 대상 역 없음")
                
        except Exception as e:
            print(f"❌ {file_path} 처리 실패: {e}")
    
    # 모든 데이터 통합
    if all_subway_data:
        final_subway_data = pd.concat(all_subway_data, ignore_index=True)
        
        print(f"\n📊 완전한 12개월 추출 결과:")
        print(f"총 데이터: {len(final_subway_data):,}개 행")
        print(f"고유 역: {final_subway_data['역명'].nunique()}개")
        print(f"고유 행정동: {final_subway_data['행정동코드'].nunique()}개")
        print(f"날짜 범위: {final_subway_data['사용일자'].min()} ~ {final_subway_data['사용일자'].max()}")
        
        # 데이터 저장
        output_file = "complete_12months_subway_data.csv"
        final_subway_data.to_csv(output_file, index=False, encoding='utf-8-sig')
        print(f"\n💾 완전한 12개월 데이터가 '{output_file}'로 저장되었습니다.")
        
        return final_subway_data
    else:
        print("❌ 추출된 데이터가 없습니다.")
        return None

# 실행
complete_subway_data = process_complete_12_months_subway_data()


🚇 완전한 12개월 지하철 데이터 처리
✅ subway/CARD_SUBWAY_MONTH_202405.csv: 1519개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202406.csv: 1470개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202407.csv: 1519개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202408.csv: 1519개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202409.csv: 1470개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202410.csv: 1519개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202411.csv: 1470개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202412.csv: 1519개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202501.csv: 1519개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202502.csv: 1372개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202503.csv: 1519개 행 추출
✅ subway/CARD_SUBWAY_MONTH_202504.csv: 1422개 행 추출

📊 완전한 12개월 추출 결과:
총 데이터: 17,837개 행
고유 역: 31개
고유 행정동: 16개
날짜 범위: 20240501 ~ 20250430

💾 완전한 12개월 데이터가 'complete_12months_subway_data.csv'로 저장되었습니다.


In [14]:
def check_subway_mapping_quality():
    """
    지하철 데이터 매핑 품질 체크
    """
    print("🔍 지하철 데이터 매핑 품질 체크")
    print("="*50)
    
    # 완전한 12개월 데이터 로드
    subway_df = pd.read_csv("complete_12months_subway_data.csv")
    
    print(f"📊 전체 데이터 현황:")
    print(f"총 데이터: {len(subway_df):,}개 행")
    print(f"고유 역: {subway_df['역명'].nunique()}개")
    print(f"고유 행정동: {subway_df['행정동코드'].nunique()}개")
    print(f"날짜 범위: {subway_df['사용일자'].min()} ~ {subway_df['사용일자'].max()}")
    
    # 1. 행정동별 매핑 현황
    print(f"\n📋 행정동별 매핑 현황:")
    admin_mapping = subway_df.groupby('행정동코드').agg({
        '역명': 'nunique',
        '사용일자': 'nunique'
    }).rename(columns={'역명': '역수', '사용일자': '일수'})
    
    for admin_code, row in admin_mapping.iterrows():
        print(f"  {admin_code}: {row['역수']}개 역, {row['일수']}일")
    
    # 2. 역별 데이터 분포
    print(f"\n📋 역별 데이터 분포:")
    station_counts = subway_df['역명'].value_counts()
    for station, count in station_counts.items():
        admin_code = subway_df[subway_df['역명'] == station]['행정동코드'].iloc[0]
        print(f"  {station} ({admin_code}): {count}개 행")
    
    # 3. 월별 데이터 분포
    print(f"\n📋 월별 데이터 분포:")
    subway_df['월'] = subway_df['사용일자'].astype(str).str[4:6]
    monthly_counts = subway_df['월'].value_counts().sort_index()
    for month, count in monthly_counts.items():
        print(f"  {month}월: {count}개 행")
    
    # 4. 데이터 품질 체크
    print(f"\n🔍 데이터 품질 체크:")
    
    # 결측치 확인
    missing_station = subway_df['역명'].isnull().sum()
    missing_admin = subway_df['행정동코드'].isnull().sum()
    missing_inflow = subway_df['하차총승객수'].isnull().sum()
    
    print(f"결측치:")
    print(f"  역명: {missing_station}개")
    print(f"  행정동코드: {missing_admin}개")
    print(f"  하차승객수: {missing_inflow}개")
    
    # 이상값 확인
    print(f"\n승객수 통계:")
    print(f"  평균 하차승객수: {subway_df['하차총승객수'].mean():.1f}명")
    print(f"  최대 하차승객수: {subway_df['하차총승객수'].max():,}명")
    print(f"  최소 하차승객수: {subway_df['하차총승객수'].min():,}명")
    
    # 5. 샘플 데이터 확인
    print(f"\n📋 샘플 데이터 (각 행정동별 1개씩):")
    sample_data = subway_df.groupby('행정동코드').first()
    print(sample_data[['역명', '사용일자', '하차총승객수']])
    
    return subway_df

# 실행
mapping_check = check_subway_mapping_quality()


🔍 지하철 데이터 매핑 품질 체크
📊 전체 데이터 현황:
총 데이터: 17,837개 행
고유 역: 31개
고유 행정동: 16개
날짜 범위: 20240501 ~ 20250430

📋 행정동별 매핑 현황:
  11110615: 4개 역, 365일
  11110650: 1개 역, 365일
  11140550: 3개 역, 365일
  11140605: 1개 역, 365일
  11170520: 3개 역, 365일
  11170650: 1개 역, 365일
  11200660: 2개 역, 365일
  11200690: 1개 역, 365일
  11440660: 1개 역, 365일
  11440690: 1개 역, 365일
  11440710: 3개 역, 365일
  11560605: 3개 역, 365일
  11620745: 2개 역, 365일
  11680510: 1개 역, 365일
  11680640: 3개 역, 365일
  11710580: 1개 역, 365일

📋 역별 데이터 분포:
  홍대입구 (11440710): 1095개 행
  종로3가 (11110615): 1095개 행
  동묘앞 (11110650): 730개 행
  여의도 (11560605): 730개 행
  당산 (11560605): 730개 행
  을지로3가 (11110615): 730개 행
  을지로4가 (11140550): 730개 행
  신촌 (11440710): 730개 행
  동대문 (11140605): 730개 행
  건대입구 (11200660): 730개 행
  선릉 (11680640): 730개 행
  합정 (11440660): 730개 행
  영등포구청 (11560605): 730개 행
  석촌 (11710580): 730개 행
  신림 (11620745): 730개 행
  삼각지 (11170520): 682개 행
  이태원 (11170650): 365개 행
  강남 (11680510): 365개 행
  용산 (11170520): 365개 행
  한강진 (11170520): 365개 행
  

In [15]:
import pandas as pd

def create_final_dataset_with_subway():
    """
    승차/하차 승객수로 최종 데이터셋 생성
    """
    print("🎯 최종 데이터셋 생성 (승차/하차 데이터 포함)")
    print("="*50)
    
    # 1. 메인 데이터셋 로드
    print("1단계: 메인 데이터셋 로드")
    main_df = pd.read_csv("dataset.csv")
    print(f"메인 데이터셋: {len(main_df):,}개 행")
    print(f"컬럼: {main_df.columns.tolist()}")
    
    # 2. 완전한 12개월 지하철 데이터 로드
    print("\n2단계: 지하철 데이터 로드")
    subway_df = pd.read_csv("complete_12months_subway_data.csv")
    print(f"지하철 데이터: {len(subway_df):,}개 행")
    print(f"고유 역: {subway_df['역명'].nunique()}개")
    print(f"고유 행정동: {subway_df['행정동코드'].nunique()}개")
    
    # 3. 지하철 데이터를 행정동별, 날짜별로 집계 (승차/하차 모두)
    print("\n3단계: 지하철 데이터 집계")
    subway_agg = subway_df.groupby(['사용일자', '행정동코드']).agg({
        '하차총승객수': 'sum',  # 유입 (해당 지역으로 들어오는 사람)
        '승차총승객수': 'sum'   # 유출 (해당 지역에서 나가는 사람)
    }).reset_index()
    
    subway_agg.rename(columns={
        '하차총승객수': 'subway_inflow',   # 유입
        '승차총승객수': 'subway_outflow'   # 유출
    }, inplace=True)
    
    print(f"집계된 지하철 데이터: {len(subway_agg):,}개 행")
    print(f"고유 날짜: {subway_agg['사용일자'].nunique()}개")
    print(f"고유 행정동: {subway_agg['행정동코드'].nunique()}개")
    
    # 4. 병합 키 생성
    print("\n4단계: 병합 키 생성")
    main_df['date_key'] = main_df['STDR_DE_ID'].astype(str)
    main_df['admin_key'] = main_df['ADSTRD_CODE_SE'].astype(str)
    subway_agg['date_key'] = subway_agg['사용일자'].astype(str)
    subway_agg['admin_key'] = subway_agg['행정동코드'].astype(str)
    
    print(f"메인 데이터 키 샘플: {main_df[['date_key', 'admin_key']].head()}")
    print(f"지하철 데이터 키 샘플: {subway_agg[['date_key', 'admin_key']].head()}")
    
    # 5. Left Join 실행
    print("\n5단계: 데이터 병합")
    merged_df = main_df.merge(
        subway_agg[['date_key', 'admin_key', 'subway_inflow', 'subway_outflow']],
        on=['date_key', 'admin_key'],
        how='left'
    )
    
    print(f"병합 후 데이터: {len(merged_df):,}개 행")
    
    # 6. 결측치 처리
    print("\n6단계: 결측치 처리")
    before_fill_inflow = merged_df['subway_inflow'].isnull().sum()
    before_fill_outflow = merged_df['subway_outflow'].isnull().sum()
    
    merged_df['subway_inflow'] = merged_df['subway_inflow'].fillna(0).astype(int)
    merged_df['subway_outflow'] = merged_df['subway_outflow'].fillna(0).astype(int)
    
    print(f"유입 결측치 처리: {before_fill_inflow}개 → 0개")
    print(f"유출 결측치 처리: {before_fill_outflow}개 → 0개")
    
    # 7. 임시 컬럼 제거
    merged_df = merged_df.drop(['date_key', 'admin_key'], axis=1)
    
    # 8. 결과 확인
    print("\n7단계: 결과 확인")
    total_records = len(merged_df)
    records_with_subway = len(merged_df[merged_df['subway_inflow'] > 0])
    
    print(f"📊 최종 병합 결과:")
    print(f"전체 레코드: {total_records:,}개")
    print(f"지하철 데이터가 있는 레코드: {records_with_subway:,}개")
    print(f"지하철 데이터 비율: {records_with_subway/total_records*100:.2f}%")
    print(f"평균 지하철 유입객: {merged_df['subway_inflow'].mean():.1f}명")
    print(f"평균 지하철 유출객: {merged_df['subway_outflow'].mean():.1f}명")
    print(f"최대 지하철 유입객: {merged_df['subway_inflow'].max():,}명")
    print(f"최대 지하철 유출객: {merged_df['subway_outflow'].max():,}명")
    
    # 9. 행정동별 지하철 이용 현황
    print(f"\n📋 행정동별 지하철 이용 현황:")
    admin_stats = merged_df.groupby('ADSTRD_CODE_SE').agg({
        'subway_inflow': ['mean', 'max'],
        'subway_outflow': ['mean', 'max']
    }).round(1)
    admin_stats.columns = ['평균유입', '최대유입', '평균유출', '최대유출']
    print(admin_stats)
    
    # 10. 최종 데이터 저장
    print("\n8단계: 데이터 저장")
    output_file = "final_dataset_with_subway.csv"
    merged_df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"💾 최종 데이터셋이 '{output_file}'로 저장되었습니다.")
    
    # 11. 최종 컬럼 확인
    print(f"\n📋 최종 데이터셋 컬럼 ({len(merged_df.columns)}개):")
    for i, col in enumerate(merged_df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    # 12. 샘플 데이터
    print(f"\n📋 최종 데이터 샘플:")
    sample_cols = ['STDR_DE_ID', 'ADSTRD_CODE_SE', 'TOT_LVPOP_CO', 'subway_inflow', 'subway_outflow', 'is_holiday', 'has_culture_event']
    print(merged_df[sample_cols].head(10))
    
    return merged_df

# 실행
print("🚀 최종 데이터셋 생성 시작!")
final_dataset = create_final_dataset_with_subway()

if final_dataset is not None:
    print("\n✅ 모든 처리가 완료되었습니다!")
    print(f"최종 데이터셋 크기: {final_dataset.shape}")
    print("📈 생활인구 예측을 위한 완전한 데이터셋이 준비되었습니다!")
    print("\n🎯 새로 추가된 feature:")
    print("  - subway_inflow: 지하철 유입 승객수 (하차)")
    print("  - subway_outflow: 지하철 유출 승객수 (승차)")


🚀 최종 데이터셋 생성 시작!
🎯 최종 데이터셋 생성 (승차/하차 데이터 포함)
1단계: 메인 데이터셋 로드
메인 데이터셋: 5,840개 행
컬럼: ['STDR_DE_ID', 'ADSTRD_CODE_SE', 'TOT_LVPOP_CO', 'culture_event_count', '분류', '유무료', 'has_culture_event', 'is_holiday', 'avg_temp', 'max_temp', 'min_temp', 'precipitation', 'wind_speed', 'sunshine_hours', 'humidity', 'ground_temp']

2단계: 지하철 데이터 로드
지하철 데이터: 17,837개 행
고유 역: 31개
고유 행정동: 16개

3단계: 지하철 데이터 집계
집계된 지하철 데이터: 5,840개 행
고유 날짜: 365개
고유 행정동: 16개

4단계: 병합 키 생성
메인 데이터 키 샘플:    date_key admin_key
0  20240501  11110615
1  20240501  11110650
2  20240501  11140550
3  20240501  11140605
4  20240501  11170520
지하철 데이터 키 샘플:    date_key admin_key
0  20240501  11110615
1  20240501  11110650
2  20240501  11140550
3  20240501  11140605
4  20240501  11170520

5단계: 데이터 병합
병합 후 데이터: 5,840개 행

6단계: 결측치 처리
유입 결측치 처리: 0개 → 0개
유출 결측치 처리: 0개 → 0개

7단계: 결과 확인
📊 최종 병합 결과:
전체 레코드: 5,840개
지하철 데이터가 있는 레코드: 5,840개
지하철 데이터 비율: 100.00%
평균 지하철 유입객: 71652.6명
평균 지하철 유출객: 69750.5명
최대 지하철 유입객: 237,061명
최대 지하철 유출객: 206,455명

📋 행정동별 지

#### 버스

🔍 7자리 코드 매칭 시도
조사 대상 8자리 코드: 16개
조사 대상 7자리 코드: 16개

📊 7자리 매칭 결과:
매칭된 데이터: 8,701개 행
매칭된 고유 7자리 코드: 7개

✅ 매칭된 7자리 코드 (7개):
  1111061 → ['11110615']
  1111065 → ['11110650']
  1114060 → ['11140605']
  1117052 → ['11170520']
  1117065 → ['11170650']
  1120066 → ['11200660']
  1120069 → ['11200690']

❌ 매칭 안된 7자리 코드 (9개):
  1114055 → ['11140550']
  1144066 → ['11440660']
  1144069 → ['11440690']
  1144071 → ['11440710']
  1156060 → ['11560605']
  1162074 → ['11620745']
  1168051 → ['11680510']
  1168064 → ['11680640']
  1171058 → ['11710580']



🗺️ 확장된 매핑 테이블 생성
🔍 매칭 안된 행정동 심층 분석
📋 버스 데이터의 모든 고유 행정동 코드:

🔍 1114055 관련 코드 찾기:
  📋 6자리 유사: ['1114059', '11140590']...
  📋 5자리 유사: ['1114059', '11140590', '1114060', '11140600', '1114061']...

🔍 1144066 관련 코드 찾기:

🔍 1144069 관련 코드 찾기:

🔍 1144071 관련 코드 찾기:

🔍 1156060 관련 코드 찾기:

🔍 1162074 관련 코드 찾기:

🔍 1168051 관련 코드 찾기:

🔍 1168064 관련 코드 찾기:

🔍 1171058 관련 코드 찾기:

🔍 특별 패턴 분석:
강남구(1168) 관련 코드: []
마포구(1144) 관련 코드: []
영등포구(1156) 관련 코드: []
기존 매핑: 7개
확장 후 매핑: 7개
